#12 bin köpek ve kedi verisi içeren bir dataset kullanarak görselin köpek mi kedi mi olduğunu tahmin etmek için kullandığım cnn mimarisi

In [2]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator


drive dan verileri çekme

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


training set veri ön işleme

In [5]:
train_datagen = ImageDataGenerator(
    rescale =1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)
training_set=train_datagen.flow_from_directory(
    '/content/drive/MyDrive/Classroom/single_prediction/training_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)


Found 8020 images belonging to 2 classes.


test seti ön işleme

In [6]:
test_datagen= ImageDataGenerator(rescale=1./255)
test_set=test_datagen.flow_from_directory(
    '/content/drive/MyDrive/Classroom/test_set',
    target_size=(64,64),
    batch_size=32,
    class_mode='binary'
)


Found 2000 images belonging to 2 classes.


cnn modeli oluşturma

In [7]:
cnn=tf.keras.models.Sequential()

convolution

In [8]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu',input_shape=[64,64,3]))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


pooling

In [9]:
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))#pool_size 2x2 matris şeklinde fmap oluşturacak

ikinci convolution katman

In [10]:
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2,strides=2))

flattening

In [11]:
cnn.add(tf.keras.layers.Flatten())

full connection

In [12]:
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

çıktı katmanı

In [13]:
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

CNN training

compile cnn

In [14]:
cnn.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [15]:
cnn.fit(x=training_set,validation_data=test_set,epochs=25)

/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 1660s 6s/step - accuracy: 0.5228 - loss: 0.7205 - val_accuracy: 0.6515 - val_loss: 0.6301
Epoch 2/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 89s 355ms/step - accuracy: 0.6370 - loss: 0.6447 - val_accuracy: 0.6975 - val_loss: 0.5885
Epoch 3/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 100s 397ms/step - accuracy: 0.6826 - loss: 0.6033 - val_accuracy: 0.7195 - val_loss: 0.5656
Epoch 4/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 89s 354ms/step - accuracy: 0.7074 - loss: 0.5622 - val_accuracy: 0.7295 - val_loss: 0.5422
Epoch 5/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 99s 394ms/step - accuracy: 0.7302 - loss: 0.5319 - val_accuracy: 0.7550 - val_loss: 0.4999
Epoch 6/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 88s 349ms/step - accuracy: 0.7442 - loss: 0.5097 - val_accuracy: 0.7665 - val_loss: 0.4951
Epoch 7/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 90s 358ms/step - accuracy: 0.7660 - loss: 0.4829 - val_accuracy: 0.7710 - val_loss: 0.4804
Epoch 8/25
251/251 ━━━━━━━━━━━━━━━━━━━━ 90s 357ms/step - accuracy: 0.7634 - loss: 0

#örnek tahmin etme

tensorflow.keras.preprocessing.image modülü, resim verisi okumak, yeniden boyutlandırmak, tensöre dönüştürmek vb. için yardımcı fonksiyonlar içerir.

load_img(path, target_size) fonksiyonu, diskten bir resmi açar.

path: Yüklemek istediğin resmin dosya yolu. (Burada uzantı .jgp olarak geçilmiş; muhtemelen .jpg olacak.)

target_size: Resmi modelin beklediği boyuta (64×64 piksel) ölçekler. Böylece ağın girdi boyutuna uygun hale gelir.

img_to_array fonksiyonu, PIL Image objesini (yüklenen resmi) 3 boyutlu bir NumPy dizisine çevirir:

Modeller genellikle birden fazla görüntüyü aynı anda işleyebilmek için “batch” boyutu bekler.

expand_dims(..., axis=0) ile dizinin en başına bir öğe daha ekler, böylece şekil (1, 64, 64, 3) olur:

1 = bu batch içinde tek bir resim var.

result dizisi boyutu (1, 1) veya (1, 2) olabilir; modele bağlı olarak farklılık gösterir. Örneğin binary sınıflama için (1,1).

.class_indices özelliği, eğitimde kullanılan klasör adlarını (örneğin "cats", "dogs") integer etiketlere nasıl eşlediğini gösteren bir sözlüktür.

Örnek çıktı: {'cats': 0, 'dogs': 1}

result[0][0]: Batch’in ilk elemanının (tek resmin) model çıktısı.

Eğitimde "dogs" etiketi için 1, "cats" etiketi için 0 atandıysa:

== 1 ise model bu resmin köpek olduğunu öngörür.

Aksi hâlde kedi olarak sınıflandırır.

In [19]:
import numpy as np
from tensorflow.keras.preprocessing import image

test_image=image.load_img('/content/drive/MyDrive/Classroom/single_prediction/cat_or_dog_1.jpg',target_size=(64,64))
test_image=image.img_to_array(test_image)
test_image=np.expand_dims(test_image,axis=0)
result=cnn.predict(test_image)
training_set.class_indices

if result[0][0] ==1:
  prediction='dog'
else:
  prediction='cat'

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 65ms/step


“Batch” terimi, derin öğrenme modellerinde aynı anda işlenen örnekler kümesini ifade eder. Bir seferde birden fazla resimle (veya veri noktasına) ileri besleme (forward pass) yapabilmek için veriler “küme” (“batch”) halinde modele verilir. Böylece model:

Veri Verimliliği ve Hız: Aynı anda, örneğin 32 veya 64 resim üzerinde hesaplama yaparak GPU’nun paralel hesaplama gücünü daha iyi kullanır.

İstikrarlı Öğrenme: Her adımda tek bir örnek yerine bir küme üzerinden gradien hesapladığında, öğrenme daha dengeli ve stabil olur.

Özetle, batch boyutu modele “kaç örnekle aynı anda” işlem yapacağını söyleyen ilk boyuttur. Tek resim için bile bu boyutun sağlanması gerekir.

In [18]:
print(prediction)

dog
